# IST 718: Big Data Analytics

- Professor: Daniel Acuna <deacuna@syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers either from your classmates or from the internet__
- You can put the homework files anywhere you want in your http://notebook.acuna.io workspace but _do not change_ the file names. The TAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and TAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before downloading and submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`). 
- Good luck!

In [ ]:
# Load the packages needed for this part
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification, pipeline, evaluation
from pyspark.sql import functions as fn, Row
from pyspark import sql

import matplotlib.pyplot as plt
import pandas as pd

# Part 2

In this section, you are going to develop a SMS spam detector based on logistic regression. This is the same idea behind sentiment analysis, but instead of predicting positive sentiment vs negative sentiment, you are going to predict whether a SMS text is spam or not.

The dataset will be in `sms_spam_df`

In [ ]:
sms_spam_df = spark.read.csv('sms_spam.csv', header=True, inferSchema=True)

# Question 2.1

Encode the `type` column to be 1 for `spam` and 0 for `ham` and store the result in `sms_spam2_df`. Besides, assign the count of spam to `spam_count` and the count of ham to `ham_count`. 

In [ ]:
# create sms_spam2_df below
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# check here
sms_spam2_df.show()

In [ ]:
# (5 pts)
np.testing.assert_array_equal(
    sms_spam2_df.groupBy('type').count().orderBy('type').rdd.map(lambda x: x['count']).collect(),
    [4827, 747])
np.testing.assert_array_equal(spam_count, 747)
np.testing.assert_array_equal(ham_count, 4827)

# Question 2.2: tfidf feature engineering
Create a pipeline that combines a `Tokenizer`, `CounterVectorizer`, and a `IDF` estimator to compute the tfidf vectors of each SMS. Fit this pipeline and assign the pipeline transformer to a variable `tfidf_pipeline`. The `Tokenizer` step should create a column `words`, the `CounterVectorizer` step should create a column `tf`, and the `IDF` step should create a column `tfidf`.

In [ ]:
# create a Pipeline transformer and name it tfidf_pipeline
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Check pipeline result
tfidf_pipeline.transform(sms_spam2_df).show()

In [ ]:
# (5 pts)
np.testing.assert_array_equal([type(s) for s in tfidf_pipeline.stages],
                              [feature.Tokenizer, feature.CountVectorizerModel, feature.IDFModel])

# Question 2.3: uppercase feature

Typical spam messages contain words that are upper case. Create a dataframe `sms_spam3_df` where you add a new column `has_uppercase` which contains an integer `1` if the first sequence of uppercase letters is longer or equal to 3 and an integer `0` otherwise. You can extract sequence of 3 or more uppercase letters by using the regular expression `[A-Z]{3,}`. You will use the function `fn.regexp_extract` to find those sequences and extract the first one (e.g., with index 0) and then use `fn.length` to compute the length of such sequence.

In [ ]:
# create sms_spam3_df below
# YOUR CODE HERE
raise NotImplementedError()

The first three messages with `has_uppercase == 1` are as follows:

```python
sms_spam3_df.where('has_uppercase == 1').take(3)
```

```console
[Row(type=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', has_uppercase=1),
 Row(type=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030', has_uppercase=1),
 Row(type=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info', has_uppercase=1)]
```

In [ ]:
# try it here
sms_spam3_df.where('has_uppercase == 1').take(3)

In [ ]:
# (5 pts)
np.testing.assert_equal(set(sms_spam3_df.columns), {'has_uppercase', 'text', 'type'})
np.testing.assert_equal(type(sms_spam3_df.schema['has_uppercase'].dataType), sql.types.IntegerType)
np.testing.assert_equal(sms_spam3_df.rdd.map(lambda x : x['has_uppercase']).sum(), 891)

# Question 2.4: Compare models

Using the following splits:

In [ ]:
training_df, validation_df, testing_df = sms_spam2_df.randomSplit([0.6, 0.3, 0.1], seed=0)

In [ ]:
[training_df.count(), validation_df.count(), testing_df.count()]

**(15 pts)** Create pipelines where the first stage is the `tfidf_pipeline` created above and the second stage is a `LogisticRegression` model with different regularization parameters ($\lambda$) and elastic net mixture ($\alpha$) as following:
$$\lambda = [0, 0.02, 0.1] $$
$$\alpha = [0.2, 0.4] $$

Try different combinations of $\lambda$ and $\alpha$ (6 combinations) to find out best parameters by using the area under the curve as the estimator. Fit those pipelines to the appropriate data split and assign the pipeline with the best model to a variable `best_model`. Also, assign $\lambda$ and $\alpha$ of the best model to `best_model_lambda` and `best_model_alpha`.

For example, the AUC on training of the first model is perfect:

```
evaluator.evaluate(lr_pipeline1.transform(training_df))
```

```console
1.0
```

In [ ]:
# Testing all the combinations of alpha and lambda and assign the best one to `best_model` based on AUC.
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# (15 pts)
np.testing.assert_equal(type(best_model), pipeline.PipelineModel)

np.testing.assert_array_equal([type(s) for s in best_model.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])

np.testing.assert_equal(best_model_lambda, 0.02)
np.testing.assert_equal(best_model_alpha, 0.2)

# Question 2.5: Generalization of best model

Using the right split and the best model selected before, compute the generalization performance and assign it to a variable `AUC_best`

In [ ]:
# assign to AUC_best the AUC of the best model selected before
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# (5 pts)
np.testing.assert_approx_equal(AUC_best, 0.9883924843423813, significant=2)

Using the same split and the best model, compute and assign `precision`, `recall` and `f1_score`. You should first count the numbers in the confusion matrix, and then compute these metrics based on the formula.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# (5 pts)
np.testing.assert_array_almost_equal([precision, recall, f1_score],
    [1.0, 0.7976190476190477, 0.8874172185430463], decimal=2)

# Question 2.6: Inference

Use the best pipeline fitted above (`best_model`) to create Pandas dataframes that contain the most negative words and the most positive words. In particular, create a dataframe `positive_words` with the columns `word` and `weight` with the top 20 positive words, sorted by descending coefficient. Similarly create a `negative_words` Pandas dataframe with the top 20 negative words where the coefficient are sorted in ascending order. **Hint: follow the `sentiment_analysis.ipynb` notebook in the repo**

In [ ]:
# create positive_words and negative_words pandas dataframe below
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# examine positive vocabulary
positive_words.head()

In [ ]:
# examine solutions
negative_words.head()

The `positive_words` and `negative_words` dataframe should look like this:

```python
positive_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>7263</th>
      <td>sexy?</td>
      <td>0.642738</td>
    </tr>
    <tr>
      <th>3555</th>
      <td>widelive.com/index.</td>
      <td>0.588182</td>
    </tr>
    <tr>
      <th>15</th>
      <td>call</td>
      <td>0.537161</td>
    </tr>
    <tr>
      <th>12237		</th>
      <td>08714712388</td>
      <td>0.504090</td>
    </tr>
    <tr>
      <th>81</th>
      <td>txt</td>
      <td>0.495005</td>
    </tr>
  </tbody>
</table>

and 

```python
negative_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>i</td>
      <td>-0.183463</td>
    </tr>
    <tr>
      <th>3332</th>
      <td>lose.</td>
      <td>-0.074937</td>
    </tr>
    <tr>
      <th>3371</th>
      <td>fightng</td>
      <td>-0.074937</td>
    </tr>
    <tr>
      <th>3221</th>
      <td>dificult</td>
      <td>-0.074937</td>
    </tr>
    <tr>
      <th>13</th>
      <td>me</td>
      <td>-0.065904</td>
    </tr>
  </tbody>
</table>

In [ ]:
# (5 pts)
np.testing.assert_equal(set(positive_words.columns), {'weight', 'word'})
np.testing.assert_equal(set(negative_words.columns), {'weight', 'word'})
np.testing.assert_approx_equal(positive_words.weight.sum(), 8.675741267346245, significant=1)
np.testing.assert_approx_equal(negative_words.weight.sum(), -0.7292131526997235, significant=1)
np.testing.assert_array_less(positive_words.weight.iloc[-1], positive_words.weight.iloc[0])
np.testing.assert_array_less(negative_words.weight.iloc[0], negative_words.weight.iloc[-1])

# Question 2.7
Use the dataframe `sms_spam3_df` to create a model where the first feature is `has_uppercase` and the next set of features are the tfidf of the text. Perform feature engineering in all features using a max absolute scaler ([`MaxAbsScaler`](https://spark.apache.org/docs/2.0.2/ml-features.html#maxabsscaler)). Do a logistic regression on the resulting scaled features with regularization parameter $\lambda = 0.2$ and elastic net mixture $\alpha=0.1$ for the entire data (all of `sms_spam3_df`). Since you have scaled all features to be within the same range, you can compare them. 

**(5 pts)** with code and comments, answer below

1. is `has_uppercase` a feature that is positively or negative related to an SMS being spam?
2. what is the ratio of the coefficient of `has_uppercase` to the biggest positive tfidf coefficient?

In [ ]:
# your code and comments below
# YOUR CODE HERE
raise NotImplementedError()